<a href="https://colab.research.google.com/github/enok/flowpredict/blob/main/flowpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Free Cash Flow Forecasting

<div style="text-align: right">The data will consist of quarterly reports spanning from 2012 to 2022.</div>

## 1. Downloading and loading stock market data from the CVM website.

### Companies to be analised


| Company       | Ticker    | CVM Code  |
| :---          | :----     | ----:     |
| Eletrobras    | ELET6     | 2437      |
| Cemig         | CMIG4     | 2453      |
| CPFL Energia  | CPFE3     | 18660     |
| Taesa         | TAEE11    | 20257     |
| Enev          | ENEV3     | 21237     |

In [26]:
companies = [18660, 2453, 2437, 21237, 20257]
print('companies: CPFE3(18660), CMIG4(2453), ELET6(2437), ENEV3(21237), TAEE11(20257)\n')

companies: CPFE3(18660), CMIG4(2453), ELET6(2437), ENEV3(21237), TAEE11(20257)



### Accounts selected


**BPA**

| Code       | Description                                          |
| :---       | :----                                                |
| 1.01       | Ativo Circulante                                     |
| 1.01.01    | Caixa e Equivalentes de Caixa                        |
| 1.01.03    | Contas a Receber                                     |
| 1.01.04    | Estoques                                             |
| 1.02.03.01 | Imobilizado em Operação                              |
| 1.02.03.02 | Direito de Uso em Arrendamento                       |

<br>                                                                

**BPP**

| Code       | Description                                          |
| :---       | :----                                                |
| 2.01       | Passivo Circulante                                   |
| 2.01.02    | Fornecedores                                         |
| 2.01.03    | Obrigações Fiscais                                   |
| 2.01.04    | Empréstimos e Financiamentos                         |

<br>                                                                
                                                                    
**DRE**

| Code       | Description                                          |
| :---       | :----                                                |
| 3.08       | Imposto de Renda e Contribuição Social sobre o Lucro |
| 3.11.01    | Atribuído a Sócios da Empresa Controladora           |

<br>                                                                

**DVA**
                                                     
| Code       | Description                                          |
| :---       | :----                                                |
| 7.04.01    | Depreciação, Amortização e Exaustão                  |

In [27]:
account_to_be_kept = ['1.01','1.01.01','1.01.03','1.01.04','1.02.03.01','1.02.03.02',
                    '2.01','2.01.02','2.01.03','2.01.04',
                    '3.08','3.11.01',
                    '7.04.01']
print(f'accounts selected: {account_to_be_kept}')

accounts selected: ['1.01', '1.01.01', '1.01.03', '1.01.04', '1.02.03.01', '1.02.03.02', '2.01', '2.01.02', '2.01.03', '2.01.04', '3.08', '3.11.01', '7.04.01']


### Installing libs

In [28]:
%pip install requests beautifulsoup4
%pip install chardet
%pip install files
%pip install chardet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Importing libs

In [29]:
import pandas as pd
import numpy as np
import re
import os
import requests
import zipfile
import chardet
from urllib.parse import urljoin
from bs4 import BeautifulSoup


### Setting Display

In [30]:
#pd.set_option('display.max_columns', 10)
pd.reset_option('display.max_columns')
pd.set_option('display.width', 200)
#pd.reset_option('display.width')

### Downloading balance sheets files

In [31]:
# Function to check if a file's encoding is UTF-8
def is_utf8(file_path):
    try:
        with open(file_path, 'rb') as file:
            raw_data = file.read()
            result = chardet.detect(raw_data)
            return result['encoding'] == 'utf-8'
    except Exception:
        return False

# URL of the website containing the ZIP files
base_url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/"

# Create a directory to save the downloaded files
download_dir = "downloaded_files"
os.makedirs(download_dir, exist_ok=True)

# Define the years you want to download (2012 to 2022)
years_to_download = set(str(year) for year in range(2012, 2023))

# Send an HTTP GET request to the URL
response = requests.get(base_url)
response.raise_for_status()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all links on the page
links = soup.find_all("a")

# Iterate through the links and download ZIP files for the specified years
for link in links:
    file_url = urljoin(base_url, link["href"])

    # Example: Assuming ZIP files are named like "itr_cia_aberta_YEAR.zip"
    if file_url.endswith(".zip"):
        zip_file_name = os.path.basename(file_url)

        # Extract the year from the ZIP file name
        year_part = zip_file_name.split("_")[-1].split(".")[0]

        # Check if the ZIP file is from a year within the specified range
        if year_part in years_to_download:
            # Check if the ZIP file already exists
            if not os.path.exists(os.path.join(download_dir, zip_file_name)):
                print(f"Downloading: {zip_file_name}")
                with open(os.path.join(download_dir, zip_file_name), "wb") as file:
                    file_response = requests.get(file_url)
                    file.write(file_response.content)
            else:
                print(f"File already exists: {zip_file_name}")

# Process the downloaded ZIP files (extract, remove "ind" files, and convert)
for zip_file_name in os.listdir(download_dir):
    if zip_file_name.endswith(".zip"):
        zip_file_path = os.path.join(download_dir, zip_file_name)
        zip_subdir = os.path.splitext(zip_file_name)[0]  # Use ZIP file name without extension as subdirectory name
        zip_subdir_path = os.path.join(download_dir, zip_subdir)

        # Check if the ZIP file has already been extracted
        if not os.path.exists(zip_subdir_path):
            print(f"Unzipping: {zip_file_name} -> {zip_subdir_path}")
            with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
                zip_ref.extractall(zip_subdir_path)

            # Remove files with "ind" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "ind" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removed: {file_path}")

            # Remove files with "itr_cia_aberta_20" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "itr_cia_aberta_20" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removed: {file_path}")

            # Remove files with "itr_cia_aberta_DFC_MD_con_20" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "itr_cia_aberta_DFC_MD_con_20" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removed: {file_path}")

            # Convert CSV files from ISO-8859-1 to UTF-8 only if they are not already UTF-8
            for csv_file_name in os.listdir(zip_subdir_path):
                if csv_file_name.endswith(".csv"):
                    csv_file_path = os.path.join(zip_subdir_path, csv_file_name)
                    if not is_utf8(csv_file_path):
                        print(f"Converting encoding: {csv_file_path}")
                        with open(csv_file_path, 'r', encoding='ISO-8859-1') as source_file:
                            content = source_file.read()
                        with open(csv_file_path, 'w', encoding='utf-8') as target_file:
                            target_file.write(content)
                    else:
                        print(f"File already in UTF-8: {csv_file_path}")

print("Download, extraction, file removal, and encoding conversion completed.")


File already exists: itr_cia_aberta_2012.zip
File already exists: itr_cia_aberta_2013.zip
File already exists: itr_cia_aberta_2014.zip
File already exists: itr_cia_aberta_2015.zip
File already exists: itr_cia_aberta_2016.zip
File already exists: itr_cia_aberta_2017.zip
File already exists: itr_cia_aberta_2018.zip
File already exists: itr_cia_aberta_2019.zip
File already exists: itr_cia_aberta_2020.zip
File already exists: itr_cia_aberta_2021.zip
File already exists: itr_cia_aberta_2022.zip
Download, extraction, file removal, and encoding conversion completed.


### Creating the datasets

In [32]:
bpa_df = None
bpp_df = None
dfc_mi_df = None
dmpl_df = None
dra_df = None
dre_df = None
dva_df = None

d_parser = lambda x: pd.datetime.strptime(format='%Y-%m-%d')

# Load files into Data Frames
for dir_name in os.listdir(download_dir):
    if not dir_name.endswith(".zip"):
        dir_path = os.path.join(download_dir, dir_name)

        for csv_file_name in os.listdir(dir_path):
            csv_file_path = os.path.join(dir_path, csv_file_name)
            print("\n-------------------------------------------------------")
            print(f"Reading csv file: {csv_file_path}")

            csv_file_type = os.path.splitext(csv_file_name)[0][15:19].replace("_", "")
            print(f"CSV type: {csv_file_type}")

            local_df = pd.read_csv(csv_file_path, sep=';')

            local_df['DT_REFER'] = pd.to_datetime(local_df['DT_REFER'], format='%Y-%m-%d')
            local_df['DT_FIM_EXERC'] = pd.to_datetime(local_df['DT_FIM_EXERC'], format='%Y-%m-%d')

            print(f"Dataframe size: {len(local_df)}")

            match csv_file_type:
                case 'BPA':
                    if bpa_df is None:
                        bpa_df = local_df
                    else:
                        bpa_df = pd.concat([bpa_df, local_df])
                    print(f"Dataframe size - after concat: {len(bpa_df)}")

                case 'BPP':
                    if bpp_df is None:
                        bpp_df = local_df
                    else:
                        bpp_df = pd.concat([bpp_df, local_df])
                    print(f"Dataframe size - after concat: {len(bpp_df)}")

                case 'DFC':
                    if dfc_mi_df is None:
                        dfc_mi_df = local_df
                    else:
                        dfc_mi_df = pd.concat([dfc_mi_df, local_df])
                    print(f"Dataframe size - after concat: {len(dfc_mi_df)}")

                case 'DMPL':
                    if dmpl_df is None:
                        dmpl_df = local_df
                    else:
                        dmpl_df = pd.concat([dmpl_df, local_df])
                    print(f"Dataframe size - after concat: {len(dmpl_df)}")

                case 'DRA':
                    if dra_df is None:
                        dra_df = local_df
                    else:
                        dra_df = pd.concat([dra_df, local_df])
                    print(f"Dataframe size - after concat: {len(dra_df)}")

                case 'DRE':
                    if dre_df is None:
                        dre_df = local_df
                    else:
                        dre_df = pd.concat([dre_df, local_df])
                    print(f"Dataframe size - after concat: {len(dre_df)}")

                case 'DVA':
                    if dva_df is None:
                        dva_df = local_df
                    else:
                        dva_df = pd.concat([dva_df, local_df])
                    print(f"Dataframe size - after concat: {len(dva_df)}")

print("\nbpa_df:")
print(bpa_df.head(2))

print("\nbpp_df:")
print(bpp_df.head(2))

print("\ndfc_mi_df:")
print(dfc_mi_df.head(2))

print("\ndmpl_df:")
print(dmpl_df.head(2))

print("\ndra_df:")
print(dra_df.head(2))

print("\ndre_df:")
print(dre_df.head(2))

print("\ndva_df:")
print(dva_df.head(2))



-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_BPA_con_2012.csv
CSV type: BPA
Dataframe size: 132416
Dataframe size - after concat: 132416

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_BPP_con_2012.csv
CSV type: BPP
Dataframe size: 224976
Dataframe size - after concat: 224976

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_DFC_MI_con_2012.csv
CSV type: DFC
Dataframe size: 80864
Dataframe size - after concat: 80864

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_DMPL_con_2012.csv
CSV type: DMPL
Dataframe size: 458552
Dataframe size - after concat: 458552

-------------------------------------------------------
Reading csv file: downloaded_files/itr_cia_aberta_2012/itr_cia_a

### Saving Dataframes

In [33]:
print("\n")
def saveToFile(dfName, df):
    fileName = dfName + '.csv'
    df = df.set_index('DT_REFER')
    print(f'\nsaving file: {fileName}')
    print(df.head(2))
    df.to_csv('processed_files/' + fileName, sep=';')

saveToFile('bpa_df', bpa_df)
saveToFile('bpp_df', bpp_df)
saveToFile('dfc_mi_df', dfc_mi_df)
saveToFile('dmpl_df', dmpl_df)
saveToFile('dra_df', dra_df)
saveToFile('dre_df', dre_df)
saveToFile('dva_df', dva_df)




saving file: bpa_df.csv
                      CNPJ_CIA  VERSAO                             DENOM_CIA  CD_CVM                                   GRUPO_DFP MOEDA ESCALA_MOEDA ORDEM_EXERC DT_FIM_EXERC CD_CONTA     DS_CONTA  \
DT_REFER                                                                                                                                                                                             
2012-03-31  00.001.180/0001-26       2  CENTRAIS ELET BRAS S.A. - ELETROBRAS    2437  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL   PENÚLTIMO   2011-12-31        1  Ativo Total   
2012-03-31  00.001.180/0001-26       2  CENTRAIS ELET BRAS S.A. - ELETROBRAS    2437  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL      ÚLTIMO   2012-03-31        1  Ativo Total   

               VL_CONTA ST_CONTA_FIXA  
DT_REFER                               
2012-03-31  163142432.0             S  
2012-03-31  164768886.0             S  

saving file: bpp_df

## 2. Data analising

### Loading Dataframes

In [34]:
print("\n")
def readCsv(dfName, df):
    fileName = dfName + '.csv'
    print(f'\nreading file: {fileName}')
    df_read = pd.read_csv('processed_files/' + fileName, sep=';')
    print(df_read.head(2))
    return df_read

bpa_df = readCsv('bpa_df', bpa_df)
bpp_df = readCsv('bpp_df', bpp_df)
dfc_mi_df = readCsv('dfc_mi_df', dfc_mi_df)
dmpl_df = readCsv('dmpl_df', dmpl_df)
dra_df = readCsv('dra_df', dra_df)
dre_df = readCsv('dre_df', dre_df)
dva_df = readCsv('dva_df', dva_df)




reading file: bpa_df.csv
     DT_REFER            CNPJ_CIA  VERSAO                             DENOM_CIA  CD_CVM                                   GRUPO_DFP MOEDA ESCALA_MOEDA ORDEM_EXERC DT_FIM_EXERC CD_CONTA     DS_CONTA  \
0  2012-03-31  00.001.180/0001-26       2  CENTRAIS ELET BRAS S.A. - ELETROBRAS    2437  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL   PENÚLTIMO   2011-12-31        1  Ativo Total   
1  2012-03-31  00.001.180/0001-26       2  CENTRAIS ELET BRAS S.A. - ELETROBRAS    2437  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL      ÚLTIMO   2012-03-31        1  Ativo Total   

      VL_CONTA ST_CONTA_FIXA  
0  163142432.0             S  
1  164768886.0             S  

reading file: bpp_df.csv
     DT_REFER            CNPJ_CIA  VERSAO                             DENOM_CIA  CD_CVM                                     GRUPO_DFP MOEDA ESCALA_MOEDA ORDEM_EXERC DT_FIM_EXERC CD_CONTA  \
0  2012-03-31  00.001.180/0001-26       2  CENTRAIS ELET BR

### Describing data structures

In [35]:
def print_df(df_name, df):
    print(f'\n\n{df_name}: {len(df):,} records')
    print('-------------------------------------------------------------------')
    print('\t\t\t TYPES')
    print(df.dtypes)

    print('\t\t\t NULL VALUES')
    print('CNPJ_CIA: ' + str(df['CNPJ_CIA'].isnull().sum().sum()))
    print('DT_REFER: ' + str(df['DT_REFER'].isnull().sum().sum()))
    print('VERSAO: ' + str(df['VERSAO'].isnull().sum().sum()))
    print('DENOM_CIA: ' + str(df['DENOM_CIA'].isnull().sum().sum()))
    print('CD_CVM: ' + str(df['CD_CVM'].isnull().sum().sum()))
    print('GRUPO_DFP: ' + str(df['GRUPO_DFP'].isnull().sum().sum()))
    print('MOEDA: ' + str(df['MOEDA'].isnull().sum().sum()))
    print('ESCALA_MOEDA: ' + str(df['ESCALA_MOEDA'].isnull().sum().sum()))
    print('ORDEM_EXERC: ' + str(df['ORDEM_EXERC'].isnull().sum().sum()))
    print('DT_FIM_EXERC: ' + str(df['DT_FIM_EXERC'].isnull().sum().sum()))
    print('CD_CONTA: ' + str(df['CD_CONTA'].isnull().sum().sum()))
    print('DS_CONTA: ' + str(df['DS_CONTA'].isnull().sum().sum()))
    print('VL_CONTA: ' + str(df['VL_CONTA'].isnull().sum().sum()))
    print('ST_CONTA_FIXA: ' + str(df['ST_CONTA_FIXA'].isnull().sum().sum()))

    print('\t\t\t COUNT UNIQUE VALUES')
    print('CNPJ_CIA: ' + str(len(df['CNPJ_CIA'].unique())))
    print('DT_REFER: ' + str(len(df['DT_REFER'].unique())))
    print('VERSAO: ' + str(len(df['VERSAO'].unique())))
    print('DENOM_CIA: ' + str(len(df['DENOM_CIA'].unique())))
    print('CD_CVM: ' + str(len(df['CD_CVM'].unique())))
    print('GRUPO_DFP: ' + str(len(df['GRUPO_DFP'].unique())))
    print('MOEDA: ' + str(len(df['MOEDA'].unique())))
    print('ESCALA_MOEDA: ' + str(len(df['ESCALA_MOEDA'].unique())))
    print('ORDEM_EXERC: ' + str(len(df['ORDEM_EXERC'].unique())))
    print('DT_FIM_EXERC: ' + str(len(df['DT_FIM_EXERC'].unique())))
    print('CD_CONTA: ' + str(len(df['CD_CONTA'].unique())))
    print('DS_CONTA: ' + str(len(df['DS_CONTA'].unique())))
    print('VL_CONTA: ' + str(len(df['VL_CONTA'].unique())))
    print('ST_CONTA_FIXA: ' + str(len(df['ST_CONTA_FIXA'].unique())))

    print('\t\t\t UNIQUE VALUES')
    print('CNPJ_CIA: ' + str(len(df['CNPJ_CIA'].unique())))
    print('DT_REFER: ' + str(len(df['DT_REFER'].unique())))
    print('VERSAO: ' + str(df['VERSAO'].unique()))
    print('DENOM_CIA: ' + str(len(df['DENOM_CIA'].unique())))
    print('CD_CVM: ' + str(len(df['CD_CVM'].unique())))
    print('GRUPO_DFP: ' + str(df['GRUPO_DFP'].unique()))
    print('MOEDA: ' + str(df['MOEDA'].unique()))
    print('ESCALA_MOEDA: ' + str(df['ESCALA_MOEDA'].unique()))
    print('ORDEM_EXERC: ' + str(df['ORDEM_EXERC'].unique()))
    print('DT_FIM_EXERC: ' + str(len(df['DT_FIM_EXERC'].unique())))
    print('CD_CONTA: ' + str(len(df['CD_CONTA'].unique())))
    print('DS_CONTA: ' + str(len(df['DS_CONTA'].unique())))
    print('VL_CONTA: ' + str(len(df['VL_CONTA'].unique())))
    print('ST_CONTA_FIXA: ' + str(df['ST_CONTA_FIXA'].unique()))


print_df('bpa_df', bpa_df)
print_df('bpp_df', bpp_df)
print_df('dfc_mi_df', dfc_mi_df)
print_df('dmpl_df', dmpl_df)
print_df('dra_df', dra_df)
print_df('dre_df', dre_df)
print_df('dva_df', dva_df)




bpa_df: 1,529,932 records
-------------------------------------------------------------------
			 TYPES
DT_REFER          object
CNPJ_CIA          object
VERSAO             int64
DENOM_CIA         object
CD_CVM             int64
GRUPO_DFP         object
MOEDA             object
ESCALA_MOEDA      object
ORDEM_EXERC       object
DT_FIM_EXERC      object
CD_CONTA          object
DS_CONTA          object
VL_CONTA         float64
ST_CONTA_FIXA     object
dtype: object
			 NULL VALUES
CNPJ_CIA: 0
DT_REFER: 0
VERSAO: 0
DENOM_CIA: 0
CD_CVM: 0
GRUPO_DFP: 0
MOEDA: 0
ESCALA_MOEDA: 0
ORDEM_EXERC: 0
DT_FIM_EXERC: 0
CD_CONTA: 0
DS_CONTA: 0
VL_CONTA: 0
ST_CONTA_FIXA: 0
			 COUNT UNIQUE VALUES
CNPJ_CIA: 634
DT_REFER: 63
VERSAO: 6
DENOM_CIA: 676
CD_CVM: 636
GRUPO_DFP: 1
MOEDA: 1
ESCALA_MOEDA: 2
ORDEM_EXERC: 2
DT_FIM_EXERC: 78
CD_CONTA: 406
DS_CONTA: 5305
VL_CONTA: 240482
ST_CONTA_FIXA: 2
			 UNIQUE VALUES
CNPJ_CIA: 634
DT_REFER: 63
VERSAO: [2 1 3 4 5 6]
DENOM_CIA: 676
CD_CVM: 636
GRUPO_DFP: ['DF Cons

## 3. Data cleansing

### Cleaning up

In [36]:
# calculate value according to field ESCALA_MOELA
def set_vl_conta(df_name, df):
    print(f'setting vl_conta based on escala_moeda: {df_name}')
    df['VL_CONTA'] = np.where(df['ESCALA_MOEDA'] == 'MIL', df['VL_CONTA'] * 1000, df['VL_CONTA'])
    return df

def remove_non_mandatory_columns(df_name, df):
    print(f'removing non mandatory columns from: {df_name}')
    existing_cols = df.columns.intersection(['CNPJ_CIA', 'VERSAO', 'DENOM_CIA', 'MOEDA', 'ESCALA_MOEDA', 'DT_INI_EXERC', 'COLUNA_DF', 'ST_CONTA_FIXA'])
    return df.drop(columns=existing_cols, axis=1)

def remove_non_used_accounts(df_name, df):
    print(f'removing non used accounts for: {df_name}')
    print(f'accounts selected: {account_to_be_kept}')
    return df[df['CD_CONTA'].isin(account_to_be_kept)]



print("\n")
bpa_df = set_vl_conta('bpa_df', bpa_df)
bpp_df = set_vl_conta('bpp_df', bpp_df)
dfc_mi_df = set_vl_conta('dfc_mi_df', dfc_mi_df)
dmpl_df = set_vl_conta('dmpl_df', dmpl_df)
dra_df = set_vl_conta('dra_df', dra_df)
dre_df = set_vl_conta('dre_df', dre_df)
dva_df = set_vl_conta('dva_df', dva_df)

print("\n")
bpa_df = remove_non_mandatory_columns('bpa_df', bpa_df)
bpp_df = remove_non_mandatory_columns('bpp_df', bpp_df)
dfc_mi_df = remove_non_mandatory_columns('dfc_mi_df', dfc_mi_df)
dmpl_df = remove_non_mandatory_columns('dmpl_df', dmpl_df)
dra_df = remove_non_mandatory_columns('dra_df', dra_df)
dre_df = remove_non_mandatory_columns('dre_df', dre_df)
dva_df = remove_non_mandatory_columns('dva_df', dva_df)

print("\n")
bpa_df = remove_non_used_accounts('bpa_df', bpa_df)
bpp_df = remove_non_used_accounts('bpp_df', bpp_df)
dfc_mi_df = remove_non_used_accounts('dfc_mi_df', dfc_mi_df)
dmpl_df = remove_non_used_accounts('dmpl_df', dmpl_df)
dra_df = remove_non_used_accounts('dra_df', dra_df)
dre_df = remove_non_used_accounts('dre_df', dre_df)
dva_df = remove_non_used_accounts('dva_df', dva_df)




setting vl_conta based on escala_moeda: bpa_df
setting vl_conta based on escala_moeda: bpp_df
setting vl_conta based on escala_moeda: dfc_mi_df
setting vl_conta based on escala_moeda: dmpl_df
setting vl_conta based on escala_moeda: dra_df
setting vl_conta based on escala_moeda: dre_df
setting vl_conta based on escala_moeda: dva_df


removing non mandatory columns from: bpa_df
removing non mandatory columns from: bpp_df
removing non mandatory columns from: dfc_mi_df
removing non mandatory columns from: dmpl_df
removing non mandatory columns from: dra_df
removing non mandatory columns from: dre_df
removing non mandatory columns from: dva_df


removing non used accounts for: bpa_df
accounts selected: ['1.01', '1.01.01', '1.01.03', '1.01.04', '1.02.03.01', '1.02.03.02', '2.01', '2.01.02', '2.01.03', '2.01.04', '3.08', '3.11.01', '7.04.01']
removing non used accounts for: bpp_df
accounts selected: ['1.01', '1.01.01', '1.01.03', '1.01.04', '1.02.03.01', '1.02.03.02', '2.01', '2.01.02', '2.

## 4. Data transformation

### Merge dataframes into one

In [37]:
df = pd.concat([bpa_df, bpp_df, dfc_mi_df, dmpl_df, dra_df, dre_df, dva_df])

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()

DT_REFER         object
CD_CVM            int64
GRUPO_DFP        object
ORDEM_EXERC      object
DT_FIM_EXERC     object
CD_CONTA         object
DS_CONTA         object
VL_CONTA        float64
dtype: object


          DT_REFER  CD_CVM                                          GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA                             DS_CONTA      VL_CONTA
2       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01                     Ativo Circulante  3.666267e+10
3       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo      ÚLTIMO   2012-03-31     1.01                     Ativo Circulante  3.665730e+10
4       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31  1.01.01        Caixa e Equivalentes de Caixa  7.994425e+09
5       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo      ÚLTIMO   2012-03-31  1.01.01        Caixa e Equivalentes 

,DT_REFER,CD_CVM,GRUPO_DFP,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
2,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01,Ativo Circulante,3.666267e+10
3,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,ÚLTIMO,2012-03-31,1.01,Ativo Circulante,3.665730e+10
4,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01.01,Caixa e Equivalentes de Caixa,7.994425e+09
5,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,ÚLTIMO,2012-03-31,1.01.01,Caixa e Equivalentes de Caixa,8.232203e+09
22,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01.03,Contas a Receber,4.352024e+09


### Saving merged Dataframes

In [38]:
saveToFile('df', df)



saving file: df.csv
            CD_CVM                                   GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA          DS_CONTA      VL_CONTA
DT_REFER                                                                                                                        
2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01  Ativo Circulante  3.666267e+10
2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo      ÚLTIMO   2012-03-31     1.01  Ativo Circulante  3.665730e+10


### Loading merged Dataframes

In [39]:
df = readCsv('df', df)

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()


reading file: df.csv
     DT_REFER  CD_CVM                                   GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA          DS_CONTA      VL_CONTA
0  2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01  Ativo Circulante  3.666267e+10
1  2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo      ÚLTIMO   2012-03-31     1.01  Ativo Circulante  3.665730e+10
DT_REFER         object
CD_CVM            int64
GRUPO_DFP        object
ORDEM_EXERC      object
DT_FIM_EXERC     object
CD_CONTA         object
DS_CONTA         object
VL_CONTA        float64
dtype: object


          DT_REFER  CD_CVM                                          GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA                             DS_CONTA      VL_CONTA
0       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01                     Ativo Circulante  3.666267e+10
1       2012-03-31    2437         DF Consolidad

,DT_REFER,CD_CVM,GRUPO_DFP,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
0,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01,Ativo Circulante,3.666267e+10
1,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,ÚLTIMO,2012-03-31,1.01,Ativo Circulante,3.665730e+10
2,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01.01,Caixa e Equivalentes de Caixa,7.994425e+09
3,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,ÚLTIMO,2012-03-31,1.01.01,Caixa e Equivalentes de Caixa,8.232203e+09
4,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01.03,Contas a Receber,4.352024e+09


### Selecting companies

In [40]:
print(f'selecting companies : {companies}')
companies_df = df[df['CD_CVM'].isin(companies)]

print(companies_df.dtypes)
print("\n")
print(companies_df)
print("\n")
companies_df.head(2)

selecting companies : [18660, 2453, 2437, 21237, 20257]
DT_REFER         object
CD_CVM            int64
GRUPO_DFP        object
ORDEM_EXERC      object
DT_FIM_EXERC     object
CD_CONTA         object
DS_CONTA         object
VL_CONTA        float64
dtype: object


          DT_REFER  CD_CVM                                          GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA                             DS_CONTA      VL_CONTA
0       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01                     Ativo Circulante  3.666267e+10
1       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo      ÚLTIMO   2012-03-31     1.01                     Ativo Circulante  3.665730e+10
2       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31  1.01.01        Caixa e Equivalentes de Caixa  7.994425e+09
3       2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo      Ú

,DT_REFER,CD_CVM,GRUPO_DFP,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
0,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01,Ativo Circulante,3.666267e+10
1,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,ÚLTIMO,2012-03-31,1.01,Ativo Circulante,3.665730e+10


### Saving companies Dataframe

In [41]:
saveToFile('../companies_df', companies_df)



saving file: ../companies_df.csv
            CD_CVM                                   GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA          DS_CONTA      VL_CONTA
DT_REFER                                                                                                                        
2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01  Ativo Circulante  3.666267e+10
2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo      ÚLTIMO   2012-03-31     1.01  Ativo Circulante  3.665730e+10


### Loading companies DataFrame

In [42]:
df = readCsv('../companies_df', companies_df)

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()


reading file: ../companies_df.csv
     DT_REFER  CD_CVM                                   GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA          DS_CONTA      VL_CONTA
0  2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01  Ativo Circulante  3.666267e+10
1  2012-03-31    2437  DF Consolidado - Balanço Patrimonial Ativo      ÚLTIMO   2012-03-31     1.01  Ativo Circulante  3.665730e+10
DT_REFER         object
CD_CVM            int64
GRUPO_DFP        object
ORDEM_EXERC      object
DT_FIM_EXERC     object
CD_CONTA         object
DS_CONTA         object
VL_CONTA        float64
dtype: object


        DT_REFER  CD_CVM                                          GRUPO_DFP ORDEM_EXERC DT_FIM_EXERC CD_CONTA                             DS_CONTA      VL_CONTA
0     2012-03-31    2437         DF Consolidado - Balanço Patrimonial Ativo   PENÚLTIMO   2011-12-31     1.01                     Ativo Circulante  3.666267e+10
1     2012-03-31    2437         DF Con

,DT_REFER,CD_CVM,GRUPO_DFP,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
0,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01,Ativo Circulante,3.666267e+10
1,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,ÚLTIMO,2012-03-31,1.01,Ativo Circulante,3.665730e+10
2,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01.01,Caixa e Equivalentes de Caixa,7.994425e+09
3,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,ÚLTIMO,2012-03-31,1.01.01,Caixa e Equivalentes de Caixa,8.232203e+09
4,2012-03-31,2437,DF Consolidado - Balanço Patrimonial Ativo,PENÚLTIMO,2011-12-31,1.01.03,Contas a Receber,4.352024e+09


### Basic functions for cleaning up

In [43]:
# Reorder the columns to move 'ORDEM_EXERC' to the last position
cols = list(df.columns)
cols.remove('ORDEM_EXERC')
cols.insert(-1, 'ORDEM_EXERC')
df = df[cols]

# Reorder the columns to move 'CD_CONTA' to the 2nd position
cols = list(df.columns)
cols.remove('CD_CONTA')
cols.insert(2, 'CD_CONTA')
df = df[cols]

# DT_REFER asdatetime
df['DT_REFER'] = pd.to_datetime(df['DT_REFER'], format='%Y-%m-%d')


def createPenultimo_DT_FIM_EXERC(df, position):

    columnName = 'DT_FIM_EXERC'

    # The field DT_FIM_EXERC for BPA and BPP is 3 months in the past, for the others (DFC_MI, DMPL, DRA, DRE, DVA) this value is past 12 months
    newColumnNameBP = columnName + '_PENULTIMO' + '_BP'
    newColumnName = columnName + '_PENULTIMO'
    bpList = ['DF Consolidado - Balanço Patrimonial Ativo', 'DF Consolidado - Balanço Patrimonial Passivo']

    df[newColumnNameBP] = np.where(
        (df['ORDEM_EXERC'] == 'PENÚLTIMO') & df['GRUPO_DFP'].isin(bpList),
        df[columnName],
        None)
    df[newColumnName] = np.where(
        (df['ORDEM_EXERC'] == 'PENÚLTIMO') & ~(df['GRUPO_DFP'].isin(bpList)),
        df[columnName],
        None)

    # remove this same values from column
    df[columnName] = np.where(
        df['ORDEM_EXERC'] != 'PENÚLTIMO',
        df[columnName],
        None)

    df[newColumnNameBP] = pd.to_datetime(df[newColumnNameBP], format='%Y-%m-%d')
    df[newColumnName] = pd.to_datetime(df[newColumnName], format='%Y-%m-%d')
    df[columnName] = pd.to_datetime(df[columnName], format='%Y-%m-%d')

    # Reorder the columns to move to correct position
    cols = list(df.columns)

        # BP
    cols.remove(newColumnNameBP)
    cols.insert(position, newColumnNameBP)
    df = df[cols]

        # not BP
    cols.remove(newColumnName)
    cols.insert(position+1, newColumnName)
    df = df[cols]

    return df

def createPenultimo_VL_CONTA(df, position):

    columnName = 'VL_CONTA'

    # The field DT_FIM_EXERC for BPA and BPP is 3 months in the past, for the others (DFC_MI, DMPL, DRA, DRE, DVA) this value is past 12 months
    newColumnName = columnName + '_PENULTIMO'

    df[newColumnName] = np.where(
        (df['ORDEM_EXERC'] == 'PENÚLTIMO'),
        df[columnName],
        None)

    # remove this same values from column
    df[columnName] = np.where(
        df['ORDEM_EXERC'] != 'PENÚLTIMO',
        df[columnName],
        None)

    df[newColumnName] = df[newColumnName].astype(float)
    df[columnName] = df[columnName].astype(float)

    # Reorder the columns to move to correct position
    cols = list(df.columns)

    cols.remove(newColumnName)
    cols.insert(position, newColumnName)
    df = df[cols]

    return df


### Create column 'DT_FIM_EXERC_PENULTIMO'

In [44]:
df = createPenultimo_DT_FIM_EXERC(df, 4)

print(df.dtypes)
print("\n")
df.head()


DT_REFER                     datetime64[ns]
CD_CVM                                int64
CD_CONTA                             object
GRUPO_DFP                            object
DT_FIM_EXERC_PENULTIMO_BP    datetime64[ns]
DT_FIM_EXERC_PENULTIMO       datetime64[ns]
DT_FIM_EXERC                 datetime64[ns]
DS_CONTA                             object
ORDEM_EXERC                          object
VL_CONTA                            float64
dtype: object




,DT_REFER,CD_CVM,CD_CONTA,GRUPO_DFP,DT_FIM_EXERC_PENULTIMO_BP,DT_FIM_EXERC_PENULTIMO,DT_FIM_EXERC,DS_CONTA,ORDEM_EXERC,VL_CONTA
0,2012-03-31,2437,1.01,DF Consolidado - Balanço Patrimonial Ativo,2011-12-31,NaT,NaT,Ativo Circulante,PENÚLTIMO,3.666267e+10
1,2012-03-31,2437,1.01,DF Consolidado - Balanço Patrimonial Ativo,NaT,NaT,2012-03-31,Ativo Circulante,ÚLTIMO,3.665730e+10
2,2012-03-31,2437,1.01.01,DF Consolidado - Balanço Patrimonial Ativo,2011-12-31,NaT,NaT,Caixa e Equivalentes de Caixa,PENÚLTIMO,7.994425e+09
3,2012-03-31,2437,1.01.01,DF Consolidado - Balanço Patrimonial Ativo,NaT,NaT,2012-03-31,Caixa e Equivalentes de Caixa,ÚLTIMO,8.232203e+09
4,2012-03-31,2437,1.01.03,DF Consolidado - Balanço Patrimonial Ativo,2011-12-31,NaT,NaT,Contas a Receber,PENÚLTIMO,4.352024e+09


#### Create column 'VL_CONTA_PENULTIMO'

In [45]:
df = createPenultimo_VL_CONTA(df, 8)

print(df.dtypes)
print("\n")
df.head()

DT_REFER                     datetime64[ns]
CD_CVM                                int64
CD_CONTA                             object
GRUPO_DFP                            object
DT_FIM_EXERC_PENULTIMO_BP    datetime64[ns]
DT_FIM_EXERC_PENULTIMO       datetime64[ns]
DT_FIM_EXERC                 datetime64[ns]
DS_CONTA                             object
ORDEM_EXERC                          object
VL_CONTA_PENULTIMO                  float64
VL_CONTA                            float64
dtype: object




,DT_REFER,CD_CVM,CD_CONTA,GRUPO_DFP,DT_FIM_EXERC_PENULTIMO_BP,DT_FIM_EXERC_PENULTIMO,DT_FIM_EXERC,DS_CONTA,ORDEM_EXERC,VL_CONTA_PENULTIMO,VL_CONTA
0,2012-03-31,2437,1.01,DF Consolidado - Balanço Patrimonial Ativo,2011-12-31,NaT,NaT,Ativo Circulante,PENÚLTIMO,3.666267e+10,NaN
1,2012-03-31,2437,1.01,DF Consolidado - Balanço Patrimonial Ativo,NaT,NaT,2012-03-31,Ativo Circulante,ÚLTIMO,NaN,3.665730e+10
2,2012-03-31,2437,1.01.01,DF Consolidado - Balanço Patrimonial Ativo,2011-12-31,NaT,NaT,Caixa e Equivalentes de Caixa,PENÚLTIMO,7.994425e+09,NaN
3,2012-03-31,2437,1.01.01,DF Consolidado - Balanço Patrimonial Ativo,NaT,NaT,2012-03-31,Caixa e Equivalentes de Caixa,ÚLTIMO,NaN,8.232203e+09
4,2012-03-31,2437,1.01.03,DF Consolidado - Balanço Patrimonial Ativo,2011-12-31,NaT,NaT,Contas a Receber,PENÚLTIMO,4.352024e+09,NaN


#### Remove disposable columns

In [46]:
cols = df.columns.intersection(['GRUPO_DFP', 'DS_CONTA', 'ORDEM_EXERC'])
df = df.drop(columns=cols, axis=1)

print(df.dtypes)
print("\n")
df.head()

DT_REFER                     datetime64[ns]
CD_CVM                                int64
CD_CONTA                             object
DT_FIM_EXERC_PENULTIMO_BP    datetime64[ns]
DT_FIM_EXERC_PENULTIMO       datetime64[ns]
DT_FIM_EXERC                 datetime64[ns]
VL_CONTA_PENULTIMO                  float64
VL_CONTA                            float64
dtype: object




,DT_REFER,CD_CVM,CD_CONTA,DT_FIM_EXERC_PENULTIMO_BP,DT_FIM_EXERC_PENULTIMO,DT_FIM_EXERC,VL_CONTA_PENULTIMO,VL_CONTA
0,2012-03-31,2437,1.01,2011-12-31,NaT,NaT,3.666267e+10,NaN
1,2012-03-31,2437,1.01,NaT,NaT,2012-03-31,NaN,3.665730e+10
2,2012-03-31,2437,1.01.01,2011-12-31,NaT,NaT,7.994425e+09,NaN
3,2012-03-31,2437,1.01.01,NaT,NaT,2012-03-31,NaN,8.232203e+09
4,2012-03-31,2437,1.01.03,2011-12-31,NaT,NaT,4.352024e+09,NaN


#### Group by 'DT_REFER', 'CD_CVM', and 'CD_CONTA' and aggregate the values

In [47]:
agg_funcs = {
    'DT_FIM_EXERC_PENULTIMO_BP': 'first',
    'DT_FIM_EXERC_PENULTIMO': 'first',
    'DT_FIM_EXERC': 'first',
    'VL_CONTA_PENULTIMO': 'last',
    'VL_CONTA': 'last'
}

df = df.groupby(['DT_REFER', 'CD_CVM', 'CD_CONTA']).agg(agg_funcs).reset_index()

# Reorder the columns as needed
df = df[['DT_REFER', 'CD_CVM', 'CD_CONTA', 'DT_FIM_EXERC_PENULTIMO_BP', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC', 'VL_CONTA_PENULTIMO', 'VL_CONTA']]

# View the final DataFrame
print(df.dtypes)
print("\n")
df.head()

DT_REFER                     datetime64[ns]
CD_CVM                                int64
CD_CONTA                             object
DT_FIM_EXERC_PENULTIMO_BP    datetime64[ns]
DT_FIM_EXERC_PENULTIMO       datetime64[ns]
DT_FIM_EXERC                 datetime64[ns]
VL_CONTA_PENULTIMO                  float64
VL_CONTA                            float64
dtype: object




,DT_REFER,CD_CVM,CD_CONTA,DT_FIM_EXERC_PENULTIMO_BP,DT_FIM_EXERC_PENULTIMO,DT_FIM_EXERC,VL_CONTA_PENULTIMO,VL_CONTA
0,2012-03-31,2437,1.01,2011-12-31,NaT,2012-03-31,3.666267e+10,3.665730e+10
1,2012-03-31,2437,1.01.01,2011-12-31,NaT,2012-03-31,7.994425e+09,8.232203e+09
2,2012-03-31,2437,1.01.03,2011-12-31,NaT,2012-03-31,4.352024e+09,4.438938e+09
3,2012-03-31,2437,1.01.04,2011-12-31,NaT,2012-03-31,7.473870e+08,7.683250e+08
4,2012-03-31,2437,1.02.03.01,2011-12-31,NaT,2012-03-31,3.522009e+10,3.600937e+10


#### Saving df before transposing

In [48]:
saveToFile('../companies_before_transposing_df', df)



saving file: ../companies_before_transposing_df.csv
            CD_CVM CD_CONTA DT_FIM_EXERC_PENULTIMO_BP DT_FIM_EXERC_PENULTIMO DT_FIM_EXERC  VL_CONTA_PENULTIMO      VL_CONTA
DT_REFER                                                                                                                   
2012-03-31    2437     1.01                2011-12-31                    NaT   2012-03-31        3.666267e+10  3.665730e+10
2012-03-31    2437  1.01.01                2011-12-31                    NaT   2012-03-31        7.994425e+09  8.232203e+09


#### Loading df before transposing

In [72]:
df = readCsv('../companies_before_transposing_df', companies_df)

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()



reading file: ../companies_before_transposing_df.csv
     DT_REFER  CD_CVM CD_CONTA DT_FIM_EXERC_PENULTIMO_BP DT_FIM_EXERC_PENULTIMO DT_FIM_EXERC  VL_CONTA_PENULTIMO      VL_CONTA
0  2012-03-31    2437     1.01                2011-12-31                    NaN   2012-03-31        3.666267e+10  3.665730e+10
1  2012-03-31    2437  1.01.01                2011-12-31                    NaN   2012-03-31        7.994425e+09  8.232203e+09
DT_REFER                      object
CD_CVM                         int64
CD_CONTA                      object
DT_FIM_EXERC_PENULTIMO_BP     object
DT_FIM_EXERC_PENULTIMO        object
DT_FIM_EXERC                  object
VL_CONTA_PENULTIMO           float64
VL_CONTA                     float64
dtype: object


        DT_REFER  CD_CVM    CD_CONTA DT_FIM_EXERC_PENULTIMO_BP DT_FIM_EXERC_PENULTIMO DT_FIM_EXERC  VL_CONTA_PENULTIMO      VL_CONTA
0     2012-03-31    2437        1.01                2011-12-31                    NaN   2012-03-31        3.666267e+10  

,DT_REFER,CD_CVM,CD_CONTA,DT_FIM_EXERC_PENULTIMO_BP,DT_FIM_EXERC_PENULTIMO,DT_FIM_EXERC,VL_CONTA_PENULTIMO,VL_CONTA
0,2012-03-31,2437,1.01,2011-12-31,NaN,2012-03-31,3.666267e+10,3.665730e+10
1,2012-03-31,2437,1.01.01,2011-12-31,NaN,2012-03-31,7.994425e+09,8.232203e+09
2,2012-03-31,2437,1.01.03,2011-12-31,NaN,2012-03-31,4.352024e+09,4.438938e+09
3,2012-03-31,2437,1.01.04,2011-12-31,NaN,2012-03-31,7.473870e+08,7.683250e+08
4,2012-03-31,2437,1.02.03.01,2011-12-31,NaN,2012-03-31,3.522009e+10,3.600937e+10


#### Transforming rows into column to have one register by date, company and account type

In [75]:
# Filling NaN values with empty strings
df.fillna('', inplace=True)

# Creating the pivot table
df_pivot = df.pivot_table(
    index=['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC_PENULTIMO_BP', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'],
    columns='CD_CONTA',
    values=['VL_CONTA_PENULTIMO', 'VL_CONTA'],
    aggfunc='first'
).reset_index()

# Flattening the multi-level columns
df_pivot.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_pivot.columns.values]

# Reordering the columns
ordered_columns = (
    ['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC_PENULTIMO_BP', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'] +
    [col for col in df_pivot.columns if col.startswith('VL_CONTA_PENULTIMO_')] +
    [col for col in df_pivot.columns if col.startswith('VL_CONTA_') and not col.startswith('VL_CONTA_PENULTIMO_')]
)

df_pivot = df_pivot[ordered_columns]

print("\n-------------------------------------")
print(df_pivot)
print("-------------------------------------\n")

# agg_funcs = {
#     'DT_FIM_EXERC_PENULTIMO_BP': 'last',
#     'DT_FIM_EXERC_PENULTIMO': 'first'
# }

# Group by 'DT_REFER' and 'CD_CVM' and display the first entry for each group
#grouped_df = df_pivot.groupby(['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC']).first().reset_index()
# grouped_df = df_pivot.groupby(['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC']).agg(agg_funcs).reset_index()

print("\n-------------------------------------")
print(df_pivot)
print("-------------------------------------\n")

def custom_agg(series):
    # If the dtype is datetime, check for non-NaT values
    if series.dtype == 'datetime64[ns]':
        non_nat_values = [val for val in series if not pd.isna(val)]
        return non_nat_values[0] if non_nat_values else pd.NaT
    else:
        # If it's numeric, check for non-NaN values
        if pd.api.types.is_numeric_dtype(series):
            non_nan_values = [val for val in series if not pd.isna(val)]
            return non_nan_values[0] if non_nan_values else np.nan
        # Else check for non-empty strings
        else:
            non_empty_values = [val for val in series if val != '']
            return non_empty_values[0] if non_empty_values else ''

# Create an aggregation dictionary excluding columns you're grouping by
grouping_columns = ['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC']
agg_dict = {col: custom_agg for col in ordered_columns if col not in grouping_columns}

# Group by 'DT_REFER', 'CD_CVM', and 'DT_FIM_EXERC', then aggregate
grouped_df = df_pivot.groupby(grouping_columns).agg(agg_dict).reset_index()

cols = list(grouped_df.columns)
cols.remove('DT_FIM_EXERC')
cols.insert(4, 'DT_FIM_EXERC')
grouped_df = grouped_df[cols]

print("\n-------------------------------------")
print(grouped_df)
print("-------------------------------------\n")

saveToFile('../df_pivot', grouped_df)



-------------------------------------
       DT_REFER  CD_CVM DT_FIM_EXERC_PENULTIMO_BP DT_FIM_EXERC_PENULTIMO DT_FIM_EXERC  VL_CONTA_PENULTIMO_1.01  VL_CONTA_PENULTIMO_1.01.01  VL_CONTA_PENULTIMO_1.01.03  VL_CONTA_PENULTIMO_1.01.04  \
0    2012-03-31    2437                                       2011-03-31   2012-03-31                      NaN                         NaN                         NaN                         NaN   
1    2012-03-31    2437                2011-12-31                          2012-03-31             3.666267e+10                7.994425e+09                4.352024e+09                 747387000.0   
2    2012-03-31    2453                                       2011-03-31   2012-03-31                      NaN                         NaN                         NaN                         NaN   
3    2012-03-31    2453                2011-12-31                          2012-03-31             8.531649e+09                2.862490e+09                2.998686e+09   